In [1]:
import glob
import pandas as pd
import numpy as np
import environment2
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from collections import defaultdict
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import wilcoxon
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk
import statistics

C:\Users\Nischal Aryal\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Nischal Aryal\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
env = environment2.environment2()
user_list_2D = env.user_list_2D

In [3]:
user_list_2D[0]

'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv'

In [4]:
fileNames2D=['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv']

In [5]:
fileNames2D=user_list_2D

In [6]:
fileNames2D

array(['data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_954edb7c-4eae-47ab-9338-5c5c7eccac2d.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_a6aab5f5-

In [7]:
def get_ROISubset(data):
    roi_subset=[]
    subset=0
    for i in range(len(data)):
        if data["State"][i]=='Answering':
            data["State"][i]='Sensemaking'
        if data['State'][i]=='Sensemaking':
            if (i < (len(data) - 1)) and data['State'][i+1]!= 'Sensemaking':
                roi_subset.append(subset)
                subset=subset+1
            else:
                roi_subset.append(subset)
        else:
            roi_subset.append(subset)
    return roi_subset



In [32]:
def get_state_action_loose_probabilities(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()

    # print(dataframe.head(5))
    states=['Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=1


    current_state=dataframe['State'][0]
    for i in range(1,len(dataframe)):
      if dataframe["State"][i]=='Answering':
        dataframe["State"][i]='Sensemaking'
      if dataframe["State"][i]== current_state:
        dataframe["high-level-action"][i-1]="same"
      else:
        dataframe["high-level-action"][i-1]="change"
      current_state=dataframe["State"][i]
    dataframe["high-level-action"][len(dataframe)-1]="same"


    last_nav_reward=0
    last_sense_reward=0
    last_forag_reward=0

    for i in range(len(dataframe)):


          if dataframe['State'][i]=="Navigation":
              #loose
            current_nav_reward=dataframe['NDSI'][i]
            print("Nav",last_nav_reward , current_nav_reward)
            if current_nav_reward < last_nav_reward:
                map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
            last_nav_reward = current_nav_reward
            probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

          elif dataframe['State'][i]=="Foraging":
              #loose
            current_forag_reward=dataframe['NDSI'][i]
            print("F",last_forag_reward , current_forag_reward)
            if current_forag_reward < last_forag_reward:
                 map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
            last_forag_reward = current_forag_reward
            probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])

          else:
            current_sense_reward=dataframe['NDSI'][i]
            print("S",last_sense_reward , current_sense_reward)
            if current_sense_reward < last_sense_reward:
                map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
            last_sense_reward = current_sense_reward
            probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])
          probabilities.append(probs)
    dataframe['probabilities']=probabilities
    return dataframe,map


In [33]:
user=[]
ROI_Cycle=[]
Foragingchange=[]
Foragingsame=[]
Foraging=[]
Sensemakingchange=[]
Sensemakingsame=[]
Sensemaking=[]
Navigationsame=[]
Navigationchange=[]
Navigation=[]
AverageROIReward=[]
IdleTimeROI=[]
ForagingProportionROI=[]
NavigationProportionROI=[]


output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','Foragingchange','Foragingsame','Sensemakingchange','Sensemakingsame','Navigationsame','Navigationchange','Foraging','Navigation','Sensemaking','AverageROIReward','IdleTimeROI','ForagingProportionROI','NavigationProportionROI'])
for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')



  #df['ROI_Cycle']=get_ROISubset(df)
  df['ROI_Cycle']=get_ROISubset(df)

  # print(df.ROI_Cycle.unique())
  for x in df.ROI_Cycle.unique():
    dfs,mappings=get_state_action_loose_probabilities((df.loc[df['ROI_Cycle'] == x]).reset_index(drop=True))
    print("Roi Cycle: #######################",x, " TimeSpent: ############## ", dfs['IdleTime'].sum())
    IdleTimeROI.append(dfs['IdleTime'].sum())
    AverageROIReward.append((df.loc[df['ROI_Cycle'] == x]['NDSI']*df.loc[df['ROI_Cycle'] == x]['ZoomLevel']).mean())
    all_dfs.append(dfs)
    maps.append(mappings)
    ROI_Cycle.append(x)
    user.append(u)
  for k in range(len(maps)):
      #print("Cycle: ",k, "Foragingchange: ",maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      Foragingchange.append(maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      #print("Cycle: ",k, "Foragingsame: ",maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      Foragingsame.append(maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']))
      Foraging.append((maps[k]['Foragingsame'] + maps[k]['Foragingchange']))


      #print("Cycle: ",k, "Navigationchange: ",maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      Navigationchange.append(maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      #print("Cycle: ",k, "Navigationsame: ",maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      Navigationsame.append(maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']))
      Navigation.append((maps[k]['Navigationsame'] + maps[k]['Navigationchange']))


      #print("Cycle: ",k, "Sensemakingchange: ",maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
      Sensemakingchange.append(maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
      #print("Cycle: ",k, "Sensemakingsame: ",maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
      Sensemakingsame.append(maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
      Sensemaking.append((maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))

      ForagingProportionROI.append((maps[k]['Foragingsame']+ + maps[k]['Foragingchange'])/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']+maps[k]['Navigationsame'] + maps[k]['Navigationchange']+maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))

      NavigationProportionROI.append((maps[k]['Navigationsame'] + maps[k]['Navigationchange'])/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']+maps[k]['Navigationsame'] + maps[k]['Navigationchange']+maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))

      print("##################################################")


output_dataframe['User']=user
output_dataframe['ROI_Cycle']=ROI_Cycle
output_dataframe['AverageROIReward']=AverageROIReward
output_dataframe['Foragingchange']=Foragingchange
output_dataframe['Foragingsame']=Foragingsame
output_dataframe['Foraging']=Foraging

output_dataframe['Sensemakingchange']=Sensemakingchange
output_dataframe['Sensemakingsame']=Sensemakingsame
output_dataframe['Sensemaking']=Sensemaking

output_dataframe['Navigationsame']=Navigationsame
output_dataframe['Navigationchange']=Navigationchange
output_dataframe['Navigation']=Navigation


output_dataframe['IdleTimeROI']=IdleTimeROI
output_dataframe['ForagingProportionROI']=ForagingProportionROI
output_dataframe['NavigationProportionROI']=NavigationProportionROI


########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
F 0 0.169186
F 0.169186 0.626178375
F 0.626178375 0.0
F 0.0 0.0
Nav 0 0.3336255
Nav 0.3336255 0.47314325
Nav 0.47314325 0.645461125
Nav 0.645461125 0.584113375
Nav 0.584113375 0.5151495
S 0 0.33769175
S 0.33769175 0.0
S 0.0 0.0
Roi Cycle: ####################### 0  TimeSpent: ##############  36.94200000000001
Nav 0 0.6424013749999999
Nav 0.6424013749999999 0.248563
Nav 0.248563 0.645461125
F 0 0.0
Nav 0.645461125 0.6424013749999999
Nav 0.6424013749999999 0.6113252499999999
S 0 0.0
S 0.0 0.6424013749999999
S 0.6424013749999999 0.4149435
S 0.4149435 0.0
S 0.0 0.4732314999999999
Roi Cycle: ####################### 1  TimeSpent: ##############  37.586999999999996
Nav 0 0.645461125
Nav 0.645461125 0.47314325
Nav 0.47314325 0.3336255
F 0 0.317388125
F 0.317388125 0.158515
F 0.158515 0.0
Nav 0.3336255 0.6344531250000001
Nav 0.6344531250000001 0.619396625
Nav 0.619396625 0.0
Nav 0.0 0.

In [34]:
output_dataframe.head(20)

,User,ROI_Cycle,Foragingchange,Foragingsame,Sensemakingchange,Sensemakingsame,Navigationsame,Navigationchange,Foraging,Navigation,Sensemaking,AverageROIReward,IdleTimeROI,ForagingProportionROI,NavigationProportionROI
0,userid_44968286-f204-4ad6-a9b5-d95b38e97866,0,0.333333,0.666667,0.333333,0.666667,0.500000,0.500000,3,4,3,1.111027,36.942,0.300000,0.400000
1,userid_44968286-f204-4ad6-a9b5-d95b38e97866,1,0.500000,0.500000,0.250000,0.750000,0.600000,0.400000,2,5,4,1.960864,37.587,0.181818,0.454545
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,2,0.500000,0.500000,0.500000,0.500000,0.666667,0.333333,4,6,2,1.499065,44.978,0.333333,0.500000
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,3,0.500000,0.500000,0.500000,0.500000,0.800000,0.200000,2,5,2,1.323280,12.711,0.222222,0.555556
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,4,0.500000,0.500000,0.500000,0.500000,0.625000,0.375000,2,8,2,1.135921,8.392,0.166667,0.666667
5,userid_44968286-f204-4ad6-a9b5-d95b38e97866,5,0.250000,0.750000,0.333333,0.666667,0.500000,0.500000,8,4,3,0.927988,36.139,0.533333,0.266667
6,userid_44968286-f204-4ad6-a9b5-d95b38e97866,6,0.500000,0.500000,0.500000,0.500000,0.666667,0.333333,2,3,2,1.076992,7.654,0.285714,0.428571
7,userid_6d49fab8-273b-4a91-948b-ecd14556b049,0,0.250000,0.750000,0.500000,0.500000,0.500000,0.500000,4,2,2,0.104363,40.945,0.500000,0.250000
8,userid_6d49fab8-273b-4a91-948b-ecd14556b049,1,0.500000,0.500000,0.500000,0.500000,0.600000,0.400000,2,5,2,0.995525,20.432,0.222222,0.555556
9,userid_6d49fab8-273b-4a91-948b-ecd14556b049,2,0.333333,0.666667,0.500000,0.500000,0.750000,0.250000,3,8,2,1.197733,32.504,0.230769,0.615385


In [35]:
output_dataframe.to_csv("data/NDSI-2D\\state_action_loose_probabilities.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)